---
## 1. Get the data

**Download the data by executing the code below:**

`Notes:`
* Import necessary libraries and load environment variables from a .env file to obtain AWS credentials and S3 bucket details.
* Configure AWS credentials and create an S3 client to interact with the specified bucket.
* List and download PDF files from the bucket, read and convert their content to text using PyMuPDF.
* Split the extracted text into manageable chunks using CharacterTextSplitter.
* Return the resulting text chunks for further use.


In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 
sys.path.append(project_root)

from src import data_utils

# Llamar a la función para descargar los PDFs
text_chunks = data_utils.download_pdfs_and_convert_to_text()

Failed to process nasdaq_annual_reports/.DS_Store in prefix nasdaq_annual_reports/: code=7: no objects found
Downloaded and processed: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2019.pdf
Downloaded and processed: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2020.pdf
Downloaded and processed: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2021.pdf
Downloaded and processed: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2022.pdf
Downloaded and processed: nasdaq_annual_reports/10x-genomics-inc/NASDAQ_TXG_2019.pdf
Downloaded and processed: nasdaq_annual_reports/10x-genomics-inc/NASDAQ_TXG_2020.pdf
Downloaded and processed: nasdaq_annual_reports/10x-genomics-inc/NASDAQ_TXG_2021.pdf
Downloaded and processed: nasdaq_annual_reports/111-inc/NASDAQ_YI_2018.pdf
Downloaded and processed: nasdaq_annual_reports/111-inc/NASDAQ_YI_2019.pdf
Downloaded and processed: nasdaq_annual_reports/111-inc/NASDAQ_YI_2020.pdf
Downloaded and processed: nasdaq_annual_reports/111-inc/NASDAQ_YI_2

Created a chunk of size 2634, which is longer than the specified 1000
Created a chunk of size 2589, which is longer than the specified 1000
Created a chunk of size 2562, which is longer than the specified 1000
Created a chunk of size 2538, which is longer than the specified 1000
Created a chunk of size 2650, which is longer than the specified 1000
Created a chunk of size 1845, which is longer than the specified 1000
Created a chunk of size 1939, which is longer than the specified 1000
Created a chunk of size 1919, which is longer than the specified 1000
Created a chunk of size 1936, which is longer than the specified 1000
Created a chunk of size 1112, which is longer than the specified 1000
Created a chunk of size 1924, which is longer than the specified 1000
Created a chunk of size 1411, which is longer than the specified 1000
Created a chunk of size 1915, which is longer than the specified 1000
Created a chunk of size 1992, which is longer than the specified 1000
Created a chunk of s


## 2. Normalize the data

**Create the following functions but not here in the notebook, do it in the python script called `text_normalizer.py` and import them into the notebook (this way you can build your own NLP preprocessing library). In fact, the structure of the functions is already written, you must complete them with the code that you consider necessary.**

- `remove_html_tags(text):` to remove all HTML tags that may be present in text.
- `remove_accented_chars(text):` to remove accented characters from text
- `expand_contractions(text):` to expand contractions of the type, "don't" to "do not". The contractions are already defined in the "contractions.py" file.
- `lemmatize_text(text):` to lemmatize text.
- `stem_text(text):` to apply stemming (NLTK's PorterStemmer) on text.
- `remove_special_chars(text):` to remove special characters from text.
- `remove_special_chars(text, remove_digits=True):` to remove numbers, note that it is the same function to remove special characters with the addition of an argument that enables or disables the removal of numbers.
- `remove_stopwords(text, stopwords=stop_words):` to remove stopwords from text.
- `remove_extra_new_lines(text):` to remove extra newlines from text.
- `remove_extra_whitespace(text):` to remove extra whitespaces from text.

If you want to add more features that would be great, for example you could start by removing emojis, using different stemming algorithms, etc. The more functions you have the better, remember that the texts are very varied and the preprocessing depends a lot on the source of our data.

To apply each of the functions you created and pre-process the dataset, you must use the `normalize_corpus()` function of the `text_normalizer.py` script. In this method each of the functions you wrote is called, in fact you must enable or disable what you consider necessary (at this point we leave it to your free choice, for example, you can lemmatize or apply stemming or directly not apply any of the two and so on with the rest, but that is your choice), this function simply groups the previous ones for a more simplified use.

In [7]:
from src import text_normalizer

# Texto cargado del PDF
pdf_text = """Aquí va el texto cargado del PDF."""

# Limpieza y normalización del texto
cleaned_text = text_normalizer.normalize_corpus(text_chunks)

AttributeError: 'list' object has no attribute 'lower'